In [4]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, load_metric
from transformers import BitsAndBytesConfig
import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, TaskType

import torch
import gc
gc.collect()
torch.cuda.empty_cache()

# 데이터셋 로드 및 전처리 함수
def load_custom_dataset(file_path):
    import json
    with open(file_path, 'r', encoding='utf-8') as f:
        data = [json.loads(line) for line in f]
    return data

def preprocess_function(examples):
    inputs = tokenizer(examples['context'], examples['prompt'], truncation=True, padding='max_length', max_length=512)
    inputs['label'] = [float(label) for label in examples['label']]  # 레이블을 float 타입으로 변환
    return inputs

# 새로운 데이터셋 경로
train_data_path = './data/nikluge-2022-nli-train.jsonl'
val_data_path = './data/nikluge-2022-nli-dev.jsonl'

# 데이터 로드
train_data = load_custom_dataset(train_data_path)
val_data = load_custom_dataset(val_data_path)

# 데이터셋 형식 변경
train_dataset = Dataset.from_dict({
    'context': [item['input']['context'] for item in train_data],
    'prompt': [item['input']['prompt'] for item in train_data],
    'label': [item['output'] for item in train_data]
})

val_dataset = Dataset.from_dict({
    'context': [item['input']['context'] for item in val_data],
    'prompt': [item['input']['prompt'] for item in val_data],
    'label': [item['output'] for item in val_data]
})


# 모델과 토크나이저 로드
model_name = "kimcando/sbert-kornli-knoSTS-trained"
# model_name = "ys7yoo/sentence-roberta-large-kor-sts"
# model_name = "Junmai/klue-roberta-large-copa-finetuned-v1"
# model_name = "ddobokki/klue-roberta-base-nli-sts"
# model_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
# model_name = "jhgan/ko-sbert-multitask"
# model_name = "BM-K/KoSimCSE-roberta-multitask"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)

# 어댑터 설정 및 적용
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none"
)
# model = get_peft_model(model, lora_config)

# 양자화 적용
def apply_quantization(module):
    if isinstance(module, torch.nn.Linear):
        return bnb.nn.Linear8bitLt(module.in_features, module.out_features, bias=module.bias is not None)
    for name, child in module.named_children():
        module.add_module(name, apply_quantization(child))
    return module

# model = apply_quantization(model)

# 데이터셋 전처리
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# 레이블 데이터 형식 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_safetensors=False,  # safetensors 사용 안함
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    greater_is_better=False,
    metric_for_best_model="mse",
)

# 평가지표 정의
def compute_metrics(p):
    preds = p.predictions.squeeze()
    labels = p.label_ids.squeeze()
    mse = ((preds - labels) ** 2).mean()
    return {"mse": mse}

# Trainer 정의
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
# 모델 학습
trainer.train()

# 모델 평가
trainer.evaluate()

import json
from datasets import Dataset

# 테스트 데이터 로드
test_data_path = './data/nikluge-2022-nli-test.jsonl'
test_data = load_custom_dataset(test_data_path)

# 테스트 데이터셋 형식 변경
test_dataset = Dataset.from_dict({
    'id': [item['id'] for item in test_data],
    'context': [item['input']['context'] for item in test_data],
    'prompt': [item['input']['prompt'] for item in test_data],
})

# 데이터셋 전처리 함수 재정의
def preprocess_test_function(examples):
    inputs = tokenizer(examples['context'], examples['prompt'], truncation=True, padding='max_length', max_length=512)
    return inputs

# 데이터셋 전처리
tokenized_test_dataset = test_dataset.map(preprocess_test_function, batched=True)

# 예측 수행
predictions = trainer.predict(tokenized_test_dataset)

# 결과 저장
output = []
for idx, pred in enumerate(predictions.predictions):
    output.append({
        'id': test_data[idx]['id'],
        'input': test_data[idx]['input'],
        'output': float(pred[0])  # 확신성 점수 예측
    })

# JSONL 파일로 저장
output_path = './data/certainty-predictions1.jsonl'
with open(output_path, 'w', encoding='utf-8') as file:
    for item in output:
        file.write(json.dumps(item, ensure_ascii=False) + '\n')

# 첫 5줄 출력
!head -n 5 ./data/certainty-predictions.jsonl

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at kimcando/sbert-kornli-knoSTS-trained and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 1/1 [00:00<00:00, 37.49ba/s]
/home/link/anaconda3/envs/18th/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mse
1,No log,1.819159,1.819159
2,No log,1.520563,1.520563
3,1.729300,1.452769,1.452770
4,1.729300,1.655967,1.655967


KeyboardInterrupt: 

In [11]:
# JSONL 파일로 저장
output_path = './data/ci.jsonl'
with open(output_path, 'w', encoding='utf-8') as file:
    for item in output:
        file.write(json.dumps(item, ensure_ascii=False) + '\n')

# 첫 5줄 출력
!head -n 5 ./data/certainty-predictions.jsonl

{"id": "nikluge-2022-nli-test-000001", "input": {"context": "또 은행의 예·적금 상품은 가입시점의 약정이율이 만기까지 적용되지만 저축성보험은 공시이율 적용주기에 따라 본인이 가입한 계약의 이율이 변동되므로 환급금이 달라질 수 있음을 기억해야 한다.", "prompt": "저축성 보험은 공시이율 적용 주기에 따라 본인이 가입한 계약의 이율이 변동되므로 환급금이 달라질 수 있다."}, "output": 6.259641647338867}
{"id": "nikluge-2022-nli-test-000002", "input": {"context": "반면 ‘다리다’는 ‘옷이나 천 따위의 주름이나 구김을 펴고 줄을 세우기 위해 다리미나 인두로 문지른다’란 뜻이다. 따라서 “다리미로 옷을 다리다” “다리지 않은 와이셔츠라 온통 구김살이 있다”에서 ‘다리다’ ‘다리지’처럼 쓴다. 명사 ‘다리미’ ‘다림질’이 동사 ‘다리다’에서 파생된 말이라고 기억하면 ‘달이다’와 ‘다리다’의 구분이 좀 더 쉬울 듯하다.", "prompt": "명사 ‘다리미’와 ‘다림질’은 동사 ‘다리다’에서 파생된 말이다."}, "output": 6.223532199859619}
{"id": "nikluge-2022-nli-test-000003", "input": {"context": "가스관 사업에서 남북 간 신뢰 문제가 걸림돌이라는 것은, 가스관 사업에 대한 진지한 논의는 남북 간 신뢰를 쌓는 기회가 될 수 있다는 뜻이기도 하다. 북은 그동안 어떻게든 우리를 건너뛰어 미국과 직거래해보려다 번번이 좌절했다. 북도 이제는 외부 세계와 숨통을 틔우려면 남북 관계를 어느 정도 정상 궤도로 돌려놓지 않으면 안 된다는 것을 깨달았을 것이다.", "prompt": "북이 외부 세계와 숨통을 틔우려면, 남북 관계를 어느 정도 정상 궤도로 돌려놓지 않으면 안 된다."}, "output": 6.282845973968506}
{"id": "nikluge-2022-nli-tes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
# JSONL 파일로 저장
output_path = './data/certainty-predictions-fin.jsonl'
with open(output_path, 'w', encoding='utf-8') as file:
    for item in output:
        file.write(json.dumps(item, ensure_ascii=False) + '\n')

# 첫 5줄 출력
!head -n 5 ./data/certainty-predictions.jsonl

{"id": "nikluge-2022-nli-test-000001", "input": {"context": "또 은행의 예·적금 상품은 가입시점의 약정이율이 만기까지 적용되지만 저축성보험은 공시이율 적용주기에 따라 본인이 가입한 계약의 이율이 변동되므로 환급금이 달라질 수 있음을 기억해야 한다.", "prompt": "저축성 보험은 공시이율 적용 주기에 따라 본인이 가입한 계약의 이율이 변동되므로 환급금이 달라질 수 있다."}, "output": 6.259641647338867}
{"id": "nikluge-2022-nli-test-000002", "input": {"context": "반면 ‘다리다’는 ‘옷이나 천 따위의 주름이나 구김을 펴고 줄을 세우기 위해 다리미나 인두로 문지른다’란 뜻이다. 따라서 “다리미로 옷을 다리다” “다리지 않은 와이셔츠라 온통 구김살이 있다”에서 ‘다리다’ ‘다리지’처럼 쓴다. 명사 ‘다리미’ ‘다림질’이 동사 ‘다리다’에서 파생된 말이라고 기억하면 ‘달이다’와 ‘다리다’의 구분이 좀 더 쉬울 듯하다.", "prompt": "명사 ‘다리미’와 ‘다림질’은 동사 ‘다리다’에서 파생된 말이다."}, "output": 6.223532199859619}
{"id": "nikluge-2022-nli-test-000003", "input": {"context": "가스관 사업에서 남북 간 신뢰 문제가 걸림돌이라는 것은, 가스관 사업에 대한 진지한 논의는 남북 간 신뢰를 쌓는 기회가 될 수 있다는 뜻이기도 하다. 북은 그동안 어떻게든 우리를 건너뛰어 미국과 직거래해보려다 번번이 좌절했다. 북도 이제는 외부 세계와 숨통을 틔우려면 남북 관계를 어느 정도 정상 궤도로 돌려놓지 않으면 안 된다는 것을 깨달았을 것이다.", "prompt": "북이 외부 세계와 숨통을 틔우려면, 남북 관계를 어느 정도 정상 궤도로 돌려놓지 않으면 안 된다."}, "output": 6.282845973968506}
{"id": "nikluge-2022-nli-tes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
string = torch.cuda.memory_summary(device=None, abbreviated=False)
# string을 20줄씩 끊어서 출력
string = string.split('\n')
for i in range(0, len(string), 20):
    print('\n'.join(string[i:i+20]))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 2            |        cudaMalloc retries: 2         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   3472 MiB |  12970 MiB |  67702 GiB |  67699 GiB |
|       from large pool |   3454 MiB |  12952 MiB |  67490 GiB |  67486 GiB |
|       from small pool |     17 MiB |     23 MiB |    212 GiB |    212 GiB |
|---------------------------------------------------------------------------|
| Active memory         |   3472 MiB |  12970 MiB |  67702 GiB |  67699 GiB |
|       from large pool |   3454 MiB |  12952 MiB |  67490 GiB |